<a href="https://colab.research.google.com/github/rosevinod/IR-Project-Group23-PatentPal/blob/main/Miscellaneous_Formula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ticker(Stock) History and details

In [ ]:
"""
!pip install yfinance
"""
#!pip install investpy
#!pip install yfinance

import yfinance as yf
import investpy

 

def ticker_info(stock_name):

    # find detail of stock using ticker
    stock = yf.Ticker(stock_name)

    #print(stock.info)
    # Serializing json
    print(f"{stock_name},{stock.info['sector']}")  



def ticker_history(stock_name):
        
    stock = yf.Ticker(stock_name)

    stockdata = stock.history(period='1mo')

    print(stockdata.to_markdown())


def history_data():   

    df = investpy.get_stock_historical_data(stock='TCS',
                                            country='India',
                                            from_date='01/01/2010',
                                            to_date='29/08/2022')
    print(df.tail())

index_list = [['HINDUNILVR.NS']]

for index_arry in index_list:
    
    for stock_name in index_arry:
        #print(stock_name)
        ticker_history(stock_name)
        #print("-------------------")  

| Date                |    Open |    High |     Low |   Close |           Volume |   Dividends |   Stock Splits |
|:--------------------|--------:|--------:|--------:|--------:|-----------------:|------------:|---------------:|
| 2022-08-02 00:00:00 | 2600    | 2639.1  | 2599    | 2635.25 |      1.35993e+06 |           0 |              0 |
| 2022-08-03 00:00:00 | 2615.25 | 2664    | 2597.9  | 2626.35 |      1.7679e+06  |           0 |              0 |
| 2022-08-04 00:00:00 | 2634    | 2643.45 | 2615    | 2627.95 |      1.18938e+06 |           0 |              0 |
| 2022-08-05 00:00:00 | 2638    | 2657.65 | 2627.55 | 2645.4  |      1.17667e+06 |           0 |              0 |
| 2022-08-08 00:00:00 | 2645.4  | 2658    | 2626.05 | 2650.35 |      1.02451e+06 |           0 |              0 |
| 2022-08-10 00:00:00 | 2650.35 | 2676.7  | 2631.2  | 2642.8  |      1.09895e+06 |           0 |              0 |
| 2022-08-11 00:00:00 | 2656.05 | 2667.75 | 2606.6  | 2613.7  | 810674           |      

## Entry and Exit of Stocks

In [ ]:
"""
!pip install yfinance
!pip install investpy
"""


import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import math
import investpy


def yfin_data(stock_name):

    stock = yf.Ticker(stock_name)

    df = stock.history(period='6y')
    df.drop(["Open","High","Low","Volume", "Dividends", "Stock Splits"], inplace = True, axis=1)
    
    #df = yf.download(stock_name, start='2010-07-22', end='2022-08-30', interval='1d')
    #df.drop(["Open","High","Low","Volume", "Adj Close"], inplace = True, axis=1)

    #print(df.to_markdown())
    return df

def investpy_data(stock_name):

    df = investpy.get_stock_historical_data(stock=stock_name,country='India'
    ,from_date='01/01/2010',to_date='29/08/2022')

    df.drop(["Open","High","Low","Volume", "Currency"], inplace = True, axis=1)
    return df

entry_arr = []; exit_arr = []

def find_swing_trading(stock_name,index_name):
    
    #df = investpy_data(stock_name)
    df = yfin_data(stock_name)
    """
    In pandas axis = 0 refers to horizontal axis or rows 
     axis = 1 refers to vertical axis or columns. 
    """
  

    #print(stockdata.tail())

    # simple moving average(SMA)   
    df["200dma"] = df["Close"].rolling(200).mean()
    df = df.dropna()
    #print(f'{df.to_markdown()}')
    global entry_arr, exit_arr
    entry_arr.clear(); exit_arr.clear()

    df = extract_entry_exit(df)

    print(f"Total Entry {len(entry_arr)} Total Exit {len(exit_arr)}")
    print(f"\n----------------------------------------\n")


def extract_entry_exit(df):
    
    
    df["10down"] = (df["200dma"].astype(float))*.97
    
    #df = df.reset_index()  # make sure indexes pair with number of rows

    #for index, row in df.iterrows():
        #print(row['10down'])

    # compare close and 10down - if true - write "close" in "entry" array
    

    df['entry'] = df['Close'][(df['Close'] <= df['10down'])]
    #print(df)       
       
   
    entry_date = df['entry'].first_valid_index()
    if entry_date is None:
        print(f"Entry not found")
        return

    entry_price = df._get_value(entry_date, 'entry')
    entry_arr.append(entry_price)
    
    print(f"entry {entry_date} {entry_price}")

    #print("------------------entry-----------------")
    #print(df.to_markdown())

    df.drop(df.index[0:df['entry'].index.get_loc(entry_date)], inplace=True)
    

    #df = df.dropna()

    #entry =  df['Close'].iloc[0:].values[0]

    #print(f"date: {df.index[0]} entry: {entry}")
   
    df['exit'] = df['Close'][(df['Close'].astype(float) > float(entry_price*1.05))]

    #print(df.to_markdown())
    exit_date = df['exit'].first_valid_index()
    if exit_date is None:
        print(f"Exit not found")
        return

    exit_price = df._get_value(exit_date, 'exit')
    exit_arr.append(exit_price)
    
    print(f"exit {exit_date} {exit_price}")

    #print("------------------exit-----------------")
    #print(df.to_markdown())

    df.drop(df.index[0:df['exit'].index.get_loc(exit_date)], inplace=True)
   
    how_many_days = (exit_date - entry_date).days
    stock_returns = "{:.2f}".format(((exit_price/entry_price)-1)*100) 

    print(f"Down period: {how_many_days} days, returns {stock_returns}% \n")
   

    #df = df.dropna()

    #print(f"date: {df.index[0]} exit: {entry}")

    #test = df['Close'].ge(entry*1.15)
    #print(test)
    
    if len(df) <= 0:
        print('no more data')
        return(df)
    else:
        df.drop(["10down","entry","exit"], inplace = True, axis=1)
        extract_entry_exit(df)
    
         


#,"HLL","HCLT"
test = ["TRENT.NS", "RELIANCE.NS", "SRF.NS", "PIDILITIND.NS", "DEEPAKNTR.NS", "NAVINFLUOR.NS", "BAJFINANCE.NS", "ADANITRANS.NS", "NAUKRI.NS", "NESTLEIND.NS", "TITAN.NS", "PGHH.NS", "ASTRAL.NS", "HONAUT.NS", "KEI.NS", "HINDUNILVR.NS", "BAJAJFINSV.NS", "HDFCBANK.NS", "ICICIBANK.NS", "HCLTECH.NS", "TCS.NS"]

index_list = [test]
index_names = ['nifty_index', 'nifty_fifty', 'nifty_next_fifty','nifty_midcap_fifty','nifty_smallcap_fifty']

try:    
    for index_name,index_arry in zip(index_names,index_list):
        
        for stock_name in index_arry:
            print(stock_name)
            find_swing_trading(stock_name,index_name)
            #print("-------------------")   
            #print(f"entry_arr {entry_arr} exit_arr {exit_arr}")
except:
  print("An exception occurred")





## Revest Money

In [ ]:
entry_arr =  [266.15, 267.55, 460.05, 465.05, 554.15, 562.45, 562.45, 819.85, 809.85, 818.55, 1838.3, 2306.5] 
exit_arr =   [287.1,  284.75, 485.8,  497.35, 586.55, 592.75, 603.35, 864.25, 861.65, 864.95, 2051.7]


def simple_invest():
    #global entry_arr, exit_arr

    shares = len(entry_arr)
    calc_return(shares)
   

def reinvest():
    
    cash = 0
    extra_share = 0
    shares = len(entry_arr)

    index = 0
    for entry,exit in zip(entry_arr,exit_arr):
        if index == 0: cash=exit; index+=1
                
        if entry <= cash:
            cash = cash-entry
            extra_share+=1
        else:
            cash = cash+exit
        
    shares = len(entry_arr) + extra_share
    calc_return(shares)
   

def calc_return(shares):

    exit_price = 0
    if len(entry_arr) > len(exit_arr):
        exit_price = entry_arr[-1]*1.05
    else:
        exit_price = exit_arr[-1]

    invested = sum(entry_arr)
    returns = shares*exit_price
    print(f'shares {shares}')
    print(f"return % {((returns/invested)-1)*100}")
    


simple_invest()
reinvest()



shares 12
return % 198.65582834064682
shares 17
return % 323.09575681591633


## Finds Stocks Runs ABOVE Nifty based on DMA 200

In [ ]:
"""
!pip install yfinance
"""

#!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import math

"""
*
Subject: load csv file in pandas dataframe using web link
*
"""
import pandas as pd

def read_nifty_web_data():

    # NIFTY
    base_uri = "https://www1.nseindia.com/content/indices/"


    # top 100 of large mid small
    nifty_hundreds_arr  = [f'{base_uri}ind_nifty100list.csv',f'{base_uri}ind_niftymidcap100list.csv'
    , f'{base_uri}ind_niftysmallcap100list.csv']

    #nifty_hundreds_arr  = [f'{base_uri}ind_nifty100list.csv']


    frames = []
    for nifty in nifty_hundreds_arr:
        #print(nifty)
        df = pd.read_csv(nifty)
        df['index_name'] = nifty.replace(base_uri,'')
        frames.append(df)

    df = pd.concat(frames,ignore_index=True)
    df.reset_index()
    # how to add new row in pandas dataframe
    df.loc[len(df.index)] = ['Nifty 50 Index','Index', '^NSEI','IQ','IN','Nifty50']

    #df.drop(df.index[1:99], inplace=True)
    
    #print(df.to_markdown())
    return df



def compare_stock_with_dma(csv_row):
    
    
    """
    In pandas axis = 0 refers to horizontal axis or rows 
     axis = 1 refers to vertical axis or columns. 
    """
    if csv_row['Symbol'] != '^NSEI':
        csv_row['Symbol'] = csv_row['Symbol']+'.NS'

    stock = yf.Ticker(csv_row['Symbol'])

    stockdata = stock.history(period='7y')
    
    stockdata.drop(["Open","High","Low","Volume", "Dividends", "Stock Splits"], inplace = True, axis=1)

    #print(stockdata.tail())

    # simple moving average(SMA)   
    stockdata["200dma"] = stockdata["Close"].rolling(200).mean()
    stockdata = stockdata.dropna()
    
    if len(stockdata) <= 0:
        return

    # difference of start date and end date of stock
    total_days = (stockdata.index[-1] - stockdata.index[0]).days
    
    if total_days <= 0:
        return

    stockdata['diff'] = stockdata['Close'] - stockdata['200dma']
    #print(stockdata.to_markdown())

    # get the last value of a column, excluding index
    current_close_price = math.ceil(stockdata['diff'].iloc[-1:].values[0])

        
    stock_returns = math.ceil(((stockdata['Close'].iloc[-1:].values[0]/stockdata['Close'].iloc[0:].values[0])-1)*100)
    
    #find how many days close price higher than dma, so remove all negative close prices    
    count_close_price_higher_to_dma = len(stockdata['diff'][(stockdata['diff'] > 0)].dropna())
    
    percentage = math.ceil((count_close_price_higher_to_dma/total_days)*100)   
    
   
    detail_url = f"https://www.screener.in/company/{csv_row['Symbol']}/consolidated/"
    trading_url = f"https://in.tradingview.com/chart/?symbol=NSE:{csv_row['Symbol']}"
        
    
    print(f"{csv_row['index_name']},{stockdata.index[0]},{stockdata.index[-1]},{csv_row['Symbol']},{percentage},{current_close_price},{stock_returns},{csv_row['Industry']},{csv_row['ISIN Code']},{detail_url},{trading_url}")

    del stock, stockdata
    

"""
"""
print("index_name,start_date,end_date,stock_name,pecentage_of_higher_%,current_close_price,stock_returns_%, Industry, ISIN_Code,detail_url,trading_url")

df = read_nifty_web_data()


try:
    for index, row in df.iterrows():
        #print(row['Industry'], row['Symbol']) 
        compare_stock_with_dma(row)
except Exception as e:
    print(e)







index_name,start_date,end_date,stock_name,pecentage_of_higher_%,current_close_price,stock_returns_%, Industry, ISIN_Code,detail_url,trading_url
ind_nifty100list.csv,2016-06-24 00:00:00,2022-08-30 00:00:00,ACC.NS,41,121,66,Construction Materials,INE012A01025,https://www.screener.in/company/ACC.NS/consolidated/,https://in.tradingview.com/chart/?symbol=NSE:ACC.NS
ind_nifty100list.csv,2016-06-24 00:00:00,2022-08-30 00:00:00,ADANIENT.NS,54,1133,7960,Metals & Mining,INE423A01024,https://www.screener.in/company/ADANIENT.NS/consolidated/,https://in.tradingview.com/chart/?symbol=NSE:ADANIENT.NS
ind_nifty100list.csv,2019-04-10 00:00:00,2022-08-30 00:00:00,ADANIGREEN.NS,63,483,6494,Power,INE364U01010,https://www.screener.in/company/ADANIGREEN.NS/consolidated/,https://in.tradingview.com/chart/?symbol=NSE:ADANIGREEN.NS
ind_nifty100list.csv,2016-06-24 00:00:00,2022-08-30 00:00:00,ADANIPORTS.NS,45,97,332,Services,INE742F01042,https://www.screener.in/company/ADANIPORTS.NS/consolidated/,https://in.trad

In [ ]:
nifty_fifty = [['^NSEI','ADANIPORTS.NS']	,['^NSEI','LT.NS']	,['^NSEI','INFY.NS']	,['^NSEI','BAJAJ-AUTO.NS']
	,['^NSEI','TCS.NS']	,['^NSEI','EICHERMOT.NS']	,['^NSEI','TECHM.NS']	,['^NSEI','CIPLA.NS']	,['^NSEI','POWERGRID.NS']
	,['^NSEI','BHARTIARTL.NS']	,['^NSEI','HCLTECH.NS']	,['^NSEI','ULTRACEMCO.NS']	,['^NSEI','WIPRO.NS']	,['^NSEI','UPL.NS']
	,['^NSEI','ONGC.NS']	,['^NSEI','GRASIM.NS']	,['^NSEI','DRREDDY.NS']	,['^NSEI','NESTLEIND.NS']	,['^NSEI','SHREECEM.NS']
	,['^NSEI','BRITANNIA.NS']	,['^NSEI','BPCL.NS']	,['^NSEI','ITC.NS']	,['^NSEI','HDFCBANK.NS']	,['^NSEI','AXISBANK.NS']
	,['^NSEI','HDFC.NS']	,['^NSEI','SUNPHARMA.NS']	,['^NSEI','ASIANPAINT.NS']	,['^NSEI','DIVISLAB.NS']	,['^NSEI','JSWSTEEL.NS']
	,['^NSEI','TITAN.NS']	,['^NSEI','KOTAKBANK.NS']	,['^NSEI','M&M.NS']	,['^NSEI','HINDUNILVR.NS']	,['^NSEI','HDFCLIFE.NS']
	,['^NSEI','ICICIBANK.NS']	,['^NSEI','RELIANCE.NS']	,['^NSEI','NTPC.NS']	,['^NSEI','MARUTI.NS']	,['^NSEI','SBILIFE.NS']
	,['^NSEI','SBIN.NS']	,['^NSEI','HEROMOTOCO.NS']	,['^NSEI','COALINDIA.NS']	,['^NSEI','TATASTEEL.NS']	,['^NSEI','BAJFINANCE.NS']
	,['^NSEI','HINDALCO.NS']	,['^NSEI','TATACONSUM.NS']	,['^NSEI','TATAMOTORS.NS']	,['^NSEI','BAJAJFINSV.NS']	,['^NSEI','APOLLOHOSP.NS']
	,['^NSEI','INDUSINDBK.NS ']]

nifty_fifty = [ "ADANIPORTS.NS", "LT.NS", "INFY.NS", "BAJAJ-AUTO.NS", "TCS.NS"
, "EICHERMOT.NS", "TECHM.NS", "CIPLA.NS", "POWERGRID.NS", "BHARTIARTL.NS", "HCLTECH.NS"
, "ULTRACEMCO.NS", "WIPRO.NS", "UPL.NS", "ONGC.NS", "GRASIM.NS", "DRREDDY.NS"
, "NESTLEIND.NS", "SHREECEM.NS", "BRITANNIA.NS", "BPCL.NS", "ITC.NS", "HDFCBANK.NS"
, "AXISBANK.NS", "HDFC.NS", "SUNPHARMA.NS", "ASIANPAINT.NS", "DIVISLAB.NS", "JSWSTEEL.NS"
, "TITAN.NS", "KOTAKBANK.NS", "M&M.NS", "HINDUNILVR.NS", "HDFCLIFE.NS", "ICICIBANK.NS"
, "RELIANCE.NS", "NTPC.NS", "MARUTI.NS","SBILIFE.NS", "SBIN.NS", "HEROMOTOCO.NS"
, "COALINDIA.NS", "TATASTEEL.NS", "BAJFINANCE.NS", "HINDALCO.NS", "TATACONSUM.NS"
, "TATAMOTORS.NS", "BAJAJFINSV.NS", "APOLLOHOSP.NS", "INDUSINDBK.NS" ]

nifty_next_fifty = ["ZOMATO.NS", "ADANITRANS.NS", "BANKBARODA.NS", "BERGEPAINT.NS"
, "INDIGO.NS", "PNB.NS", "PGHH.NS", "CHOLAFIN.NS", "IOC.NS", "NYKAA.NS", "PAYTM.NS"
, "NAUKRI.NS", "JUBLFOOD.NS", "AMBUJACEM.NS", "ICICIGI.NS", "BAJAJHLDNG.NS", "PIDILITIND.NS"
, "COLPAL.NS", "BOSCHLTD.NS", "HDFCAMC.NS", "ADANIENT.NS", "BIOCON.NS", "ADANIGREEN.NS"
, "GODREJCP.NS", "GLAND.NS", "ZYDUSLIFE.NS", "MARICO.NS", "ACC.NS", "HAVELLS.NS", "BANDHANBNK.NS"
, "SRF.NS", "DABUR.NS", "MUTHOOTFIN.NS", "SBICARD.NS", "SAIL.NS", "TORNTPHARM.NS"
, "NMDC.NS", "LUPIN.NS", "SIEMENS.NS", "MCDOWELL-N.NS", "PEL.NS", "VEDL.NS", "DMART.NS"
, "DLF.NS", "PIIND.NS", "MINDTREE.NS", "LTI.NS", "ICICIPRULI.NS", "INDUSTOWER.NS", "GAIL.NS" ]

nifty_midcap_fifty = ["OFSS.NS", "IDFCFIRSTB.NS", "IDEA.NS", "CANBK.NS", "FEDERALBNK.NS"
, "AUROPHARMA.NS", "LICHSGFIN.NS", "BALKRISIND.NS", "PFC.NS", "HONAUT.NS", "AUBANK.NS"
, "HINDPETRO.NS", "CONCOR.NS", "ABBOTINDIA.NS", "TVSMOTOR.NS", "RECLTD.NS", "TATACOMM.NS"
, "MRF.NS", "HAL.NS", "TATAPOWER.NS", "BATAINDIA.NS", "IRCTC.NS", "TORNTPOWER.NS", "TRENT.NS"
, "SRTRANSFIN.NS", "M&MFIN.NS", "ABB.NS", "BHARATFORG.NS", "L&TFH.NS", "ASTRAL.NS"
, "UBL.NS", "BHEL.NS", "VOLTAS.NS", "PAGEIND.NS", "ALKEM.NS", "SUNTV.NS", "BEL.NS"
, "PETRONET.NS", "GUJGASLTD.NS", "MPHASIS.NS", "LTTS.NS", "CUMMINSIND.NS", "JINDALSTEL.NS"
, "RAMCOCEM.NS", "ZEEL.NS", "MFSL.NS", "COFORGE.NS", "GODREJPROP.NS", "ASHOKLEY.NS", "ESCORTS.NS"]


nifty_smallcap_fifty = ["CHAMBLFERT.NS", "POONAWALLA.NS", "ROUTE.NS", "STLTECH.NS"
, "HINDCOPPER.NS", "UTIAMC.NS", "CYIENT.NS", "IDFC.NS", "GRAPHITE.NS", "MAHABANK.NS"
, "ZENSARTECH.NS", "AMBER.NS", "CESC.NS", "CAMS.NS", "NBCC.NS", "FSL.NS", "JBCHEPHARM.NS"
, "CANFINHOME.NS", "DBL.NS", "GRANULES.NS", "CHEMPLASTS.NS", "ANGELONE.NS", "BIRLACORPN.NS"
, "JUBLINGREA.NS", "IOB.NS", "IBULHSGFIN.NS", "RADICO.NS", "BALAMINES.NS", "MEDPLUS.NS"
, "CDSL.NS", "KPITTECH.NS", "ANURAS.NS", "HEG.NS", "ALOKINDS.NS", "AMARAJABAT.NS"
, "LXCHEM.NS", "LATENTVIEW.NS", "MCX.NS", "INTELLECT.NS", "PVR.NS", "HFCL.NS", "DEVYANI.NS"
, "METROBRAND.NS", "BSOFT.NS", "PNBHOUSING.NS", "INDIGOPNTS.NS", "WELSPUNIND.NS", "BAJAJELEC.NS"
, "CENTRALBK.NS", "RBLBANK.NS"]




## Load Multiple CSV file from Google Drive and Merge it

In [ ]:
"""
*
Subject: load csv file in pandas dataframe using google drive folder
*
"""
import pandas as pd

from google.colab import drive
#drive.mount('/content/drive/')

#%cd /content/drive/MyDrive/Colab Notebooks/NIFTY/
"""
df = pd.read_csv("ind_nifty50list.csv")
df = pd.read_csv("ind_niftynext50list.csv")
df = pd.read_csv("ind_niftymidcap50list.csv")
df = pd.read_csv("ind_niftysmallcap50list.csv")
"""

# Read CSV files from List
df = pd.concat(map(pd.read_csv, ['ind_nifty50list.csv'
, 'ind_niftynext50list.csv','ind_niftymidcap50list.csv','ind_niftysmallcap50list.csv']),ignore_index=True)
df.reset_index()
print(df.to_markdown())

## Load CSV data from NSE website , concat (merge) multiple CSV

In [ ]:
"""
*
Subject: load csv file in pandas dataframe using web link
*
"""
import pandas as pd

# NIFTY
base_uri = "https://www1.nseindia.com/content/indices/"

# top 50 of large mid small
nifty_fiftys_arr = [f'{base_uri}ind_nifty50list.csv', f'{base_uri}ind_niftynext50list.csv'
,f'{base_uri}ind_niftymidcap50list.csv',f'{base_uri}ind_niftysmallcap50list.csv']

# top 100 of large mid small
nifty_hundreds_arr  = [f'{base_uri}ind_nifty100list.csv',f'{base_uri}ind_niftymidcap100list.csv'
, f'{base_uri}ind_niftysmallcap100list.csv']

nifty_sector_arr  = [f'{base_uri}ind_niftybanklist.csv',f'{base_uri}ind_niftyconsumerdurableslist.csv'
, f'{base_uri}ind_niftyfmcglist.csv', f'{base_uri}ind_niftyhealthcarelist.csv'
, f'{base_uri}ind_niftyitlist.csv', f'{base_uri}ind_niftypharmalist.csv']



frames = []
for nifty in nifty_hundreds_arr:
    #print(nifty)
    df = pd.read_csv(nifty)
    df['index_name']=nifty.replace(base_uri,'')
    frames.append(df)

df = pd.concat(frames,ignore_index=True)
df.reset_index()
# how to add new row in pandas dataframe
df.loc[len(df.index)] = ['Nifty 50 Index','Index', '^NSEI','IQ','IN','Nifty50']

print(df.to_markdown())



|     | Company Name                                          | Industry                          | Symbol     | Series   | ISIN Code    | index_name                   |
|----:|:------------------------------------------------------|:----------------------------------|:-----------|:---------|:-------------|:-----------------------------|
|   0 | ACC Ltd.                                              | Construction Materials            | ACC        | EQ       | INE012A01025 | ind_nifty100list.csv         |
|   1 | Adani Enterprises Ltd.                                | Metals & Mining                   | ADANIENT   | EQ       | INE423A01024 | ind_nifty100list.csv         |
|   2 | Adani Green Energy Ltd.                               | Power                             | ADANIGREEN | EQ       | INE364U01010 | ind_nifty100list.csv         |
|   3 | Adani Ports and Special Economic Zone Ltd.            | Services                          | ADANIPORTS | EQ       | INE742F01042 | ind_nifty10

## Buy and Sale Stock based on today's conditions

In [ ]:
#!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import math
from datetime import datetime

cash, shares, buy_price = 10000,0,0
inflation = float((round((7/365), 2)/100))

def GetStock(ticker):

    global df

    stock = yf.Ticker(ticker)

    df = stock.history(period='5y')
    
    df.drop(["Open","High","Low","Volume", "Dividends", "Stock Splits"], inplace = True, axis=1)

    #print(stockdata.tail())

    # simple moving average(SMA)   
    df["200dma"] = df["Close"].rolling(200).mean()
    df = df.dropna()
    
    if len(df) <= 0:
        return
   
   
    while len(df) > 0:   
        open_market(df)
        print(f'df length {len(df)}')

def open_market(df):

    global partial_sold_price, stock_increaser
    partial_sold_price = 0
    stock_increaser = 1.03

    #print(df)    
    for index, row in df.iterrows():
        if cash < row['Close']:
            print('money finish')        
            df = df.drop(df.index,inplace=True)
            return
        elif row['Close'] < row['200dma']:
            Buy(df, index, row)
            break
    
    #print(f'buy price {buy_price} buy date {buy_date}')

    # delete data not useful after buying the stock
    df.drop(df.index[0:df.index.get_loc(buy_date)], inplace=True)    
   
    # sale all the shares if greater than 1, always keep 1 share in stock

    for index, row in df.iterrows(): 
        if shares <= 1:
            break
        else:
            days = (index - buy_date).days
            #print(partial_sold_price,days,inflation)
            if days > 0:
                #print(f'stock_increaser {stock_increaser}')
                partial_sold_price = ((buy_price*((days*inflation)+1))*stock_increaser)
                #print(f"price to sale {partial_sold_price} buy price {buy_price} close price {row['Close']}")
                if row['Close'] > partial_sold_price:
                    print('market high sale')
                    SalePartial(df, index, row)
                elif row['Close'] < ((buy_price*((days*inflation)+1))) and cash > row['Close']:
                    print('market down buy')
                    BuyPartial(df, index, row)
  
    print('\n----------------------\n')

def Buy(df, index, row):
    global cash,shares,buy_price, buy_date
    buy_price = float(round(row['Close'], 2))
    shares = shares + math.floor(cash/row['Close'])
    cash = cash - math.floor(shares*row['Close'])
    buy_date = index
    print(f'buying - cash,shares {cash,shares}')
    

def BuyPartial(df, index, row):
    global cash,shares,buy_price, buy_date
    #buy_price = float(round(row['Close'], 2))
    shares = shares + math.floor(cash/row['Close'])
    cash = cash - math.floor(shares*row['Close'])
    #buy_date = index
    print(f'buying - cash,shares {cash,shares}')
    
def Sale(df, index,row):
    global cash, sold_price, sold_date, shares
            
    print('sold all')
    cash = cash + (shares*row['Close'])
    sold_price = row['Close']
    sold_date = index
    shares = 0
    df.drop(df.index[0:df.index.get_loc(sold_date)], inplace=True) 

def SalePartial(df, index,row):
    global cash, shares, partial_sold_price, partial_sold_date,stock_increaser
    shares-=1
    cash = cash + (1*row['Close'])
    partial_sold_price = row['Close']
    partial_sold_date = index
    #print(f'sold price {partial_sold_price} sold date {partial_sold_date}')
    stock_increaser+=0.03
    df.drop(df.index[0:df.index.get_loc(partial_sold_date)], inplace=True)
    print(f'sold - cash,shares {cash,shares}')

GetStock("HINDUNILVR.NS")


buying - cash,shares (474, 6)
market high sale
sold - cash,shares (2123.0126953125, 5)
market down buy
buying - cash,shares (-7436.9873046875, 6)
market high sale
sold - cash,shares (-5706.022705078125, 5)
market high sale
sold - cash,shares (-3839.2998046875, 4)
market high sale
sold - cash,shares (-1906.2384033203125, 3)
market high sale
sold - cash,shares (36.963134765625, 2)
market high sale
sold - cash,shares (1993.38525390625, 1)

----------------------

df length 716
money finish
df length 0


## Buy Low & Sale High

In [ ]:
#!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import math
from datetime import datetime
import json

account = {}
account["saving_account"] = 1500
account["shares"] = 0
account["sip_date_per_month"] = datetime.fromisoformat('1996-01-01 00:00:00')
account["sip_count"] =  0
account["sip_fund"] =  0
account["last_buy_price"] = 0
inflation = 1+float((round((7/365), 2)/100))


def GetStock(ticker):

    stock = yf.Ticker(ticker)

    df = stock.history(period='6y')
    
    df.drop(["Open","High","Low","Volume", "Dividends", "Stock Splits"], inplace = True, axis=1)

    df["200dma"] = df["Close"].rolling(200).mean()
    df = df.dropna()
    
    #start SIP after 30 days 
    account["sip_date_per_month"] = df.index.values[0]

    if len(df) <= 0:
        return
 
    print(df.to_markdown())
    open_market(df)

    account["total_investment"] = account['saving_account']+account['sip_fund']
    account["total_return"] = account['saving_account']+(account['last_buy_price']*account['shares'])
    account["total_profit"] = round((((account["total_return"]/account["total_investment"])-1)*100),2)
    print(json.dumps(account, indent=2, default=str))
    #print(f'{account}')
 
def open_market(df):

    #print(df)    
    for index, row in df.iterrows():
        sip(index,row)
        if row["Close"] < row["200dma"]:                        
            if account["saving_account"] > row['Close']:
                #print("--------------------")
                Buy(index,row)
            else:
                df.drop(df.index[0:df.index.get_loc(index)], inplace=True)
        elif row["Close"] > row["200dma"] and account['shares'] > 1:
            #print("--------------------")
            Sale(index,row)    

def sip(index,row):
    # add money to saving_account
    days = (index - account["sip_date_per_month"]).days   
    if days >= 30:
        sip_value = 1000 +(days*inflation)
        account["saving_account"]+=sip_value
        account["sip_date_per_month"]=index
        account["sip_count"]+=1
        account["sip_fund"]+=sip_value
    #print(f'SIP days {days} index {index} account {account} \n')


def Buy(index, row): 
    
    #print(f'buy or stop loss - index row {index,row}')
    # if account["last_buy_price"] == 0:
    #     account["last_buy_price"] = row['Close']
    

    # if row['Close'] < account["last_buy_price"] and account["shares"] > 0:               
    #     account["saving_account"] = account["saving_account"] + math.floor(account['shares']*row['Close'])
    #     account["shares"]-=account['shares']
    #     account["last_buy_price"] = row['Close']
    #     #print(f'Stop loss {account}')        

    # else:
    #     #print(f"check balance before buy account {account} : buy rate {row['Close']}")
    #     how_many_shares = math.ceil((account["saving_account"]/row['Close'])/2)
    #     account["shares"] = account["shares"] + how_many_shares
    #     account["saving_account"] = account["saving_account"] - math.floor(how_many_shares*row['Close'])   
    #     account["last_buy_price"] = row['Close']
    #     #print(f'Buy {account}')

    #print(f"check balance before buy account {account} : buy rate {row['Close']}")
    how_many_shares = math.ceil((account["saving_account"]/row['Close'])/2)
    account["shares"] = account["shares"] + how_many_shares
    account["saving_account"] = account["saving_account"] - math.floor(how_many_shares*row['Close'])   
    account["last_buy_price"] = row['Close']
    #print(f'Buy Date/Rate {index, row["Close"]}Buy {account}')



def Sale(index,row):    
    # #print(f'sale - index row {index,row}')
    # is_profit = (row['Close']*account["shares"]) - (account["last_buy_price"]*account["shares"])
    # #print(f'profit {is_profit} close price {row["Close"]} dma {row["200dma"]}')
    # if is_profit >= row['Close']:
    #     account["saving_account"] = account["saving_account"] + row['Close']
    #     account["shares"]-=1
    
    # #print(f'Sale {account}')


    if row['Close'] > math.ceil(row['200dma']*1.03):
        account["saving_account"] = account["saving_account"] + math.floor(1*row['Close'])
        account["shares"]-=1
        account["last_buy_price"] = row['Close']
        #print(f'Sale Date/Rate {index, row["Close"]} After SALE account {account}')



GetStock("HDFCBANK.NS")


| Date                |    Close |   200dma |
|:--------------------|---------:|---------:|
| 2017-06-23 00:00:00 |  808.605 |  655.854 |
| 2017-06-27 00:00:00 |  803.306 |  656.779 |
| 2017-06-28 00:00:00 |  802.728 |  657.698 |
| 2017-06-29 00:00:00 |  806.414 |  658.598 |
| 2017-06-30 00:00:00 |  801.08  |  659.504 |
| 2017-07-03 00:00:00 |  804.256 |  660.42  |
| 2017-07-04 00:00:00 |  801.71  |  661.32  |
| 2017-07-05 00:00:00 |  799.48  |  662.235 |
| 2017-07-06 00:00:00 |  804.038 |  663.198 |
| 2017-07-07 00:00:00 |  808.426 |  664.158 |
| 2017-07-10 00:00:00 |  812.863 |  665.096 |
| 2017-07-11 00:00:00 |  814.827 |  666.063 |
| 2017-07-12 00:00:00 |  815.239 |  667.036 |
| 2017-07-13 00:00:00 |  816.354 |  668.004 |
| 2017-07-14 00:00:00 |  814.608 |  668.929 |
| 2017-07-17 00:00:00 |  815.772 |  669.846 |
| 2017-07-18 00:00:00 |  816.354 |  670.806 |
| 2017-07-19 00:00:00 |  820.597 |  671.783 |
| 2017-07-20 00:00:00 |  829.616 |  672.812 |
| 2017-07-21 00:00:00 |  825.81  |

## Compare Using Inflation

In [ ]:
#!pip install yfinance
#!pip install investpy

import yfinance as yf
import investpy
import pandas as pd
import numpy as np
import math
import json
import sys
from datetime import datetime, timedelta



def yfin_data(ticker):
   
    stock = yf.Ticker(ticker)

    df = stock.history(period='3y')
    
    df.drop(["Open","High","Low","Volume", "Dividends", "Stock Splits"], inplace = True, axis=1)

    print(df.head(),df.tail())
    return df
    
    #df = yf.download(stock_name, start='2010-07-22', end='2022-08-30', interval='1d')
    #df.drop(["Open","High","Low","Volume", "Adj Close"], inplace = True, axis=1)


def investpy_data(stock_name):

    df = investpy.get_stock_historical_data(stock=stock_name,country='India'
    ,from_date='31/08/2017',to_date='01/09/2022')

    df.drop(["Open","High","Low","Volume", "Currency"], inplace = True, axis=1)
    return df

def GetStock(ticker):

    global df
    df = investpy_data(ticker)

    first_day_price,last_day_price = df["Close"].loc[df.index.values[0]], df["Close"].loc[df.index.values[-1]] 
    #print(f'first_day_price,last_day_price {first_day_price,last_day_price}')
    
    account['stock_return_lifetime'] = ((last_day_price/first_day_price)-1)*100
    account['stock_days_lifetime'] =  (df.index[-1] - df.index[0]).days  


    #df["200dma"] = df["Close"].rolling(200).mean()    
    #df = df.dropna()
    
    if len(df) <= 0:
        return

    #start SIP after 30 days 
    account["sip_date_per_month"] = df.index.values[0]

    # saving account is equal to the price of 1st share
    account["saving_account"] = float(df.iloc[df['Close'].index.get_loc(df.index[0])])
    account["sip_amount_per_month"] = float(df.iloc[df['Close'].index.get_loc(df.index[0])])

    account["todays_closing_price"] = float(df.iloc[df['Close'].index.get_loc(df.index[-1])])

    #print(df.to_markdown())
    #print(df.head(),df.tail())
       

    open_market(df)

    account["total_investment"] = account['sip_fund']
    account["total_return"] = account['saving_account']+(account["todays_closing_price"]*account['shares'])
    account["total_profit"] = round((((account["total_return"]/account["total_investment"])-1)*100),2)
    print(json.dumps(account, indent=2, default=str))
    #print(f'{account}')
 
def open_market(df):


    #print(df)    
    for index, row in df.iterrows():
        sip(index,row)
        
        if account["saving_account"] > row['Close']  :            
            Buy(index,row)
        
        if account["shares"] > 1 :    
            Sale(index,row)    

def sip(index,row):

    # add money to saving_account
    days = (index - account["sip_date_per_month"]).days   
    if days >= 30:
        #print(f'sip date: {index}')
        sip_value = account["sip_amount_per_month"] #+(days*inflation)
        #weak_fund_value = account["sip_amount_per_month"]*.20 #+(days*inflation)
        account["saving_account"]+=sip_value
        #account["fifty_two_week_fund"]+=weak_fund_value
        account["sip_date_per_month"]=index # this used for 30 days calculation
        account["sip_count"]+=1
        account["sip_fund"] = (account["sip_fund"] + sip_value)
    #print(f'SIP days {days} index {index} account {account} \n')

def FindHighestOrLowest(index, increase_days=0, minmax='min'): 
    
    global aday
    
    back_date = index - timedelta(days=aday)
    one_day_back = index - timedelta(days=1)
    if back_date in df.index and one_day_back in df.index:   
            
        one_year_data = df.iloc[df['Close'].index.get_loc(back_date):df['Close'].index.get_loc(one_day_back)] 
        
        if len(one_year_data) > math.ceil(aday/2): 
            if minmax == 'min':                
                return one_year_data['Close'].min()
            else:                
                return one_year_data['Close'].max()
   
    return None


"""
logic - increase the share price and store it 
if selling price higher than buying sell it 
"""
def ProfitPerShare(index,row):
    global buy_history_df    
    df2 = pd.DataFrame([math.ceil(row['Close']*1.05)])
    buy_history_df = pd.concat([buy_history_df,df2],ignore_index=True)


def Buy(index, row):
    global aday   
    
    # if account["last_sold_price"] != 0:
    #     #print(f'account["last_sold_price"] {account["last_sold_price"]} row["Close"] {row["Close"]}')
    #     if account["last_sold_price"]*0.95 < row["Close"]:
    #         return

    # lowest = FindHighestOrLowest(index, 0, minmax='min')
    # #print(f"current price {row['Close']} lowest price {lowest}")
    # if lowest is not None:
    #     if row['Close'] <= lowest:
    #         #print(f'buy weak date {index} rate {row["Close"]}')
    #         BuyWeak(index, row)
    #         return

    #print(f'buy normal date {index} rate {row["Close"]}')   

    """
    buy one share at a time and sell it if profit greater than 10% per share
    """
    account["shares"]+=1
    account["saving_account"] = account["saving_account"] - math.floor(1*row['Close'])   
    account["last_buy_price"] = row['Close']
    ProfitPerShare(index,row)        
    
        
    

def BuyWeak(index, row):
    
    """
    buy more than one share at a time and sell it if profit greater than 10% per share
    """
    how_many_shares_saving = math.floor((account["saving_account"]/row['Close']))
    
    how_many_shares_weak = 0
    if account["fifty_two_week_fund"] > row['Close']:
        how_many_shares_weak = math.floor((account["fifty_two_week_fund"]/row['Close']))
        account["fifty_two_week_fund"] = account["fifty_two_week_fund"] - math.floor(how_many_shares_weak*row['Close']) 
    
    account["shares"] = account["shares"] + how_many_shares_saving + how_many_shares_weak
    account["saving_account"] = account["saving_account"] - math.floor(how_many_shares_saving*row['Close'])
     
    account["last_buy_price"] = row['Close']        
    ProfitPerShare(index,row)



def Sale(index,row):
    #global buy_history_df 
    
    #if len(buy_history_df) < 1:
        #return

    if row["Close"] > ((row["Close"]*account["shares"])*.10):
        account["saving_account"] = account["saving_account"] + row['Close']
        account["shares"]-=1

    # #print(f'sale - index row {index,row} account {account}')
    # is_profit = ((row["Close"]*(account["shares"]-1) - account['last_buy_price']*account["shares"]) )
    # if is_profit > row['Close']:    
    #     #if (row["Close"]*(account["shares"]-1)) >= buy_history_df[0].iloc[0]:
    #     account["saving_account"] = account["saving_account"] + row['Close']
    #     account["shares"]-=1
    #     buy_history_df = buy_history_df.iloc[1: , :]
    #     #sys.exit()

    # highest = FindHighestOrLowest(index, 0, minmax='max')   
    # if highest is not None:
    #     if row['Close'] >= highest:            
    #         #print(f"sale high - current date {index} current rate {row['Close']} maximum rate {df_lowest['Close'].max()}")            
    #         #print(f'sale high date {index} rate {row["Close"]}')
    #         SaleHigh(index, row)
    #         return           
    
    
    # is_profit = ((row["Close"]*(account["shares"]-1) - account['last_buy_price']*account["shares"]) )
    # #if is_profit > row['Close']:
    # # sale the stock if greater than 10% of its buying price    
    # if row["Close"] >= buy_history_df[0].iloc[0]:
    #     account["saving_account"] = account["saving_account"] + row['Close']
    #     account["shares"]-=1
    #     buy_history_df = buy_history_df.iloc[1: , :]
    #     account["last_sold_price"]=row["Close"]
    #     #print(f'sale normal date {index} rate {row["Close"]}')
    #     #sys.exit()

def SaleHigh(index,row):

    global buy_history_df    
    account["saving_account"] = account["saving_account"] + row['Close']*2
    #account["fifty_two_week_fund"] = account["fifty_two_week_fund"] + row['Close']*1
    account["shares"]-=2


search_result = investpy.search_quotes(text='idfc', products=['stocks'],
countries=['india'], n_results=1)
print(search_result)


aday = 0
days_value = [20,30,40,50,60,70,80,90,100,364,500]
days_value = [80]
for day in days_value:
    aday = day
    print(f'aday {aday}--------------')
    account.clear()
    account = {}
    account["saving_account"] = 0
    account["fifty_two_week_fund"] = 0
    account["shares"] = 0
    account["sip_date_per_month"] = datetime.fromisoformat('1996-01-01 00:00:00')
    account["sip_count"] =  0
    account["sip_fund"] =  0
    account["last_buy_price"] = 0
    account["last_sold_price"] = 0
    inflation = 1+float((round((7/365), 2)/100))
    buy_history_df = pd.DataFrame()
    GetStock("BJFN")



## Buy Normal and Sell at 10%

In [64]:
#!pip install yfinance
#!pip install investpy

import yfinance as yf
import investpy
import pandas as pd
import numpy as np
import math
import json
import sys
from datetime import datetime, timedelta



def yfin_data(ticker):
   
    stock = yf.Ticker(ticker)

    df = stock.history(period='3y')
    
    df.drop(["Open","High","Low","Volume", "Dividends", "Stock Splits"], inplace = True, axis=1)

    print(df.head(),df.tail())
    return df
    
    #df = yf.download(stock_name, start='2010-07-22', end='2022-08-30', interval='1d')
    #df.drop(["Open","High","Low","Volume", "Adj Close"], inplace = True, axis=1)


def investpy_data(stock_name):

    df = investpy.get_stock_historical_data(stock=stock_name,country='India'
    ,from_date='31/08/2015',to_date='01/09/2022')

    df.drop(["Open","High","Low","Volume", "Currency"], inplace = True, axis=1)
    return df

def GetStock(ticker):

    global df
    df = investpy_data(ticker)

    first_day_price,last_day_price = df["Close"].loc[df.index.values[0]], df["Close"].loc[df.index.values[-1]] 
    #print(f'first_day_price,last_day_price {first_day_price,last_day_price}')
    
    account['stock_days_lifetime'] =  (df.index[-1] - df.index[0]).days

    account['stock_return_lifetime'] = ((last_day_price/first_day_price)-1)*100
      
    if len(df) <= 0:
        return

    #start SIP after 30 days 
    account["sip_date_per_month"] = df.index.values[0]

    # saving account is equal to the price of 1st share
    account["saving_account"] = float(df.iloc[df['Close'].index.get_loc(df.index[0])])
    account["sip_amount_per_month"] = float(df.iloc[df['Close'].index.get_loc(df.index[0])])

    account["todays_closing_price"] = float(df.iloc[df['Close'].index.get_loc(df.index[-1])])

    #print(df.to_markdown())
    #print(df.head(),df.tail())
       

    open_market(df)

    account["total_investment"] = account['sip_fund']
    account["total_return"] = account['saving_account']+(account["todays_closing_price"]*account['shares'])
    account["total_profit"] = round((((account["total_return"]/account["total_investment"])-1)*100),2)
    print(json.dumps(account, indent=2, default=str))
    #print(f'{account}')
 
def open_market(df):


    #print(df)    
    for index, row in df.iterrows():
        sip(index,row)
        
        if account["saving_account"] > row['Close']  :            
            Buy(index,row)
        
        if account["shares"] > 1 :    
            Sale(index,row)    

def sip(index,row):

    # add money to saving_account
    days = (index - account["sip_date_per_month"]).days   
    if days >= 30:
        #print(f'sip date: {index}')
        sip_value = account["sip_amount_per_month"] #+(days*inflation)
        #weak_fund_value = account["sip_amount_per_month"]*.20 #+(days*inflation)
        account["saving_account"]+=sip_value
        #account["fifty_two_week_fund"]+=weak_fund_value
        account["sip_date_per_month"]=index # this used for 30 days calculation
        account["sip_count"]+=1
        account["sip_fund"] = (account["sip_fund"] + sip_value)
    #print(f'SIP days {days} index {index} account {account} \n')

"""
if -ve result price increase
if +ve result price decrease
"""

def HowMuchIncreaseOrDecrease(index,row): 
      
    back_date = index - timedelta(days=30)
        
    if back_date in df.index :            
        old_df = df.iloc[df['Close'].index.get_loc(back_date):df['Close'].index.get_loc(index)] 
        old_close_price = float(old_df.iloc[old_df['Close'].index.get_loc(old_df.index[0])])
        
        res = float((old_close_price/row['Close'])-1)*100
        #print(f'{res}')
        return res
    else:
        return 0

"""
buy one share at a time and sell it if profit greater than 10% per share
"""
def Buy(index, row):
    
    how_many_share_want_buy = 1

    res = HowMuchIncreaseOrDecrease(index, row)
    
    if res >= 15.0:
        #print(f'buy {res}')
        how_many_share_want_buy = 2
        if account["saving_account"] > row['Close']*how_many_share_want_buy:   
            account["shares"]+=how_many_share_want_buy
            account["saving_account"] = account["saving_account"] - math.floor(how_many_share_want_buy*row['Close'])   
            account["last_buy_price"] = row['Close']
    else:
        account["shares"]+=how_many_share_want_buy
        account["saving_account"] = account["saving_account"] - math.floor(how_many_share_want_buy*row['Close'])   
        account["last_buy_price"] = row['Close']
        


def Sale(index,row):  
    
    how_many_share_want_sell = 1

    res = HowMuchIncreaseOrDecrease(index, row)
    
    if res <= -15.0 :
        how_many_share_want_sell = 1 
        if account["shares"] > how_many_share_want_sell:   
            account["saving_account"] = account["saving_account"] + row['Close']*how_many_share_want_sell
            account["shares"]-=how_many_share_want_sell
    # elif row['Close'] > (account["shares"]*row['Close']*.10):
    #     account["saving_account"] = account["saving_account"] + row['Close']*how_many_share_want_sell
    #     account["shares"]-=how_many_share_want_sell

    
        
account = {}
account["saving_account"] = 0
account["fifty_two_week_fund"] = 0
account["shares"] = 0
account["sip_date_per_month"] = datetime.fromisoformat('1996-01-01 00:00:00')
account["sip_count"] =  0
account["sip_fund"] =  0
account["last_buy_price"] = 0
account["last_sold_price"] = 0
inflation = 1+float((round((7/365), 2)/100))
#GetStock("BJFN")
GetStock("HLL")
#GetStock("IIAN")
#GetStock("PROC")
#GetStock("SOFT")
#GetStock("NEST")
#GetStock("ABOT")
#GetStock("YESB")


{
  "saving_account": 1785.8500000000022,
  "fifty_two_week_fund": 0,
  "shares": 50,
  "sip_date_per_month": "2022-08-03 00:00:00",
  "sip_count": 83,
  "sip_fund": 71454.70000000004,
  "last_buy_price": 2593.0,
  "last_sold_price": 0,
  "stock_days_lifetime": 2558,
  "stock_return_lifetime": 202.70646997328376,
  "sip_amount_per_month": 860.9,
  "todays_closing_price": 2606.0,
  "total_investment": 71454.70000000004,
  "total_return": 132085.85,
  "total_profit": 84.85
}


## Buy & Sale Above 200DMA

In [ ]:
#!pip install yfinance
#!pip install investpy

import yfinance as yf
import investpy
import pandas as pd
import numpy as np
import math
import json
import sys
from datetime import datetime, timedelta



def yfin_data(ticker):
   
    stock = yf.Ticker(ticker)

    df = stock.history(period='7y')
    
    #df.drop(["Open","High","Low","Volume", "Dividends", "Stock Splits"], inplace = True, axis=1)

    df['Close2'] = (df['Open'] +df['High'] +df['Low'] +df['Close'])/4
    #print(df.head(),df.tail())
    #sys.exit()
    return df
    
    #df = yf.download(stock_name, start='2010-07-22', end='2022-08-30', interval='1d')
    #df.drop(["Open","High","Low","Volume", "Adj Close"], inplace = True, axis=1)


def investpy_data(stock_name):

    df = investpy.get_stock_historical_data(stock=stock_name,country='India'
    ,from_date='31/08/2015',to_date='01/09/2022')

    df.drop(["Open","High","Low","Volume", "Currency"], inplace = True, axis=1)
    return df

def GetStock(ticker):

    global df
    df = yfin_data(ticker)
    #df = investpy_data(ticker)

    df["200dma"] = df["Close"].rolling(200).mean()
    #df.dropna()
    if len(df) <= 0:
        return

    #start SIP after 30 days 
    account["sip_date_per_month"] = df.index.values[0]
    
    last_index_value = df.index[-1]
    get_array_loc = df.index.get_loc(last_index_value)
    value_at_loc_last_close_price = df['Close'].values[get_array_loc]
       
    account["todays_closing_price"] = float(value_at_loc_last_close_price)

    account['stock_days_lifetime'] =  (df.index[-1] - df.index[0]).days

    first_index_value = df.index[0]
    get_array_loc = df.index.get_loc(first_index_value)
    value_at_loc_first_close_price = df['Close'].values[get_array_loc]

    first_day_price,last_day_price = value_at_loc_first_close_price, value_at_loc_last_close_price
    account['stock_return_lifetime'] = ((last_day_price/first_day_price)-1)*100
    
    #print(df.to_markdown())
    #print(df.head(),df.tail())
       
    open_market2(df)

    account["total_investment"] = account['sip_fund']
    account["total_return"] = account['saving_account']+(account["todays_closing_price"]*account['shares'])
    account["total_profit"] = round((((account["total_return"]/account["total_investment"])-1)*100),2)
    print(json.dumps(account, indent=2, default=str))
    #print(f'{account}')
    
def open_market(df):

    #print(df)    
    for index, row in df.iterrows():
        sip(index,row)

        if row["Close"] > row["200dma"]:                        
            if account["saving_account"] > row['Close']:
                #print("--------------------")                
                Buy(index,row)                
            if account['shares'] > 0:
                #print("--------------------")
                Sale(index,row)
        else:
            account["last_sold_price"] = 0
        #df.drop(df.index[0:df.index.get_loc(index)], inplace=True)

def open_market2(df):

    #print(df)    
    for index, row in df.iterrows():
        sip(index,row)
                               
        IncreaseStockCounter(index, row)

def IncreaseStockCounter(index,row):

    if account["stock_increase_counter"] == 5:
        if account["saving_account"] > row['Close']:
            account["stock_increase_counter"] = 0
            account["stock_decrease_counter"] = 0
            Buy2(index,row)
            return
    
    if account["stock_decrease_counter"] == 5:
        if account["shares"] > 0:
            account["stock_increase_counter"] = 0
            account["stock_decrease_counter"] = 0
            Sale2(index,row)
            return

    # if account["stock_increase_counter"] == 0:
    #     account["stock_increase_counter"]+=1
    #     account["last_buy_price"] = row['Close']
    # el
    
    if row['Close'] > account["last_buy_price"]:
         account["stock_increase_counter"]+=1
         account["last_buy_price"] = row['Close']

         account["stock_decrease_counter"] = 0
         return
    
    if row['Close'] < account["last_buy_price"]:
             
        account["stock_decrease_counter"]+=1
        account["last_buy_price"] = row['Close']

        account["stock_increase_counter"] = 0
        #if account["shares"] > 0:
            #Sale2(index,row)


        

def sip(index,row):
    
    # add money to saving_account
    days = (index - account["sip_date_per_month"]).days   
    if days >= 30:
        #print(f'sip {index}')
        sip_value = 1000 #+(days*inflation)       
        account["saving_account"]+=sip_value      
        account["sip_date_per_month"]=index # this used for 30 days calculation
        account["sip_count"]+=1
        account["sip_fund"] = (account["sip_fund"] + sip_value)
   

def Buy(index, row):
    
    if account["last_sold_price"] == 0:
        if row['Close'] > row['200dma']*1.02:
            Buying(index,row)
    elif row['Close'] > account["last_sold_price"]:
        Buying(index,row)

        
def Buying(index, row):
    #print(f"BUY - close {row['Close']} index {index} dma {row['200dma']} ")
    no_of_share = math.floor(account["saving_account"]/row['Close'])   
    account["shares"]+=no_of_share
    account["saving_account"] = account["saving_account"] - (no_of_share*row['Close']) - 20 
    account["last_buy_price"] = row['Close']


def Sale(index,row):
  
    if row["Close"] < account["last_buy_price"]:
        #print(f"SALE close {row['Close']} index {index} dma {row['200dma']}")
        account["saving_account"] = account["saving_account"] + (row['Close']*account["shares"])-20
        account["shares"] = 0
        account["last_sold_price"] = row['Close']

def Buy2(index,row):
    print(f"BUY - close {row['Close']} index {index} dma {row['200dma']} ")
    no_of_share = math.floor(account["saving_account"]/row['Close'])   
    account["shares"]+=no_of_share
    account["saving_account"] = account["saving_account"] - (no_of_share*row['Close']) - 20 
    account["last_buy_price"] = row['Close']

def Sale2(index,row):
    print(f"SALE close {row['Close']} index {index} dma {row['200dma']}")
    account["saving_account"] = account["saving_account"] + (row['Close']*account["shares"])-20
    account["shares"] = 0
    account["last_sold_price"] = row['Close']


     
account = {}
account["saving_account"] = 0
account["shares"] = 0
account["sip_date_per_month"] = datetime.fromisoformat('1996-01-01 00:00:00')
account["sip_count"] =  0
account["sip_fund"] =  0
account["last_buy_price"] = 0
account["last_sold_price"] = 0
account["stock_increase_counter"] = 0
account["stock_decrease_counter"] = 0
inflation = 1+float((round((7/365), 2)/100))
#GetStock("BJFN")
#GetStock("BAJFINANCE.NS")
GetStock("HINDUNILVR.NS")
#GetStock("IIAN")
#GetStock("PROC")
#GetStock("SOFT")
#GetStock("NEST")
#GetStock("ABOT")
#GetStock("YESB")
#GetStock("HEROMOTOCO.NS")
